# 셋팅

- Mac에서는 페이지 랜더링이 매우 느림....    
    참고 자료: https://macro-maker.tistory.com/16   
    -> docker 사용해야할 듯

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from tqdm import tqdm
import pickle

def save_file_to_pickle(file:str, file_path:str):
    """
    file: 저장할 파일
    file_path: 저장할 이름 (~.pkl)
    """
    # pickle 파일로 저장
    with open(file_path, 'wb') as f:  # 'wb'는 write binary의 약자
        pickle.dump(file, f)
        
def load_file_from_pickle(file_path:str):
    # pickle 파일에서 데이터 불러오기
    with open(file_path, 'rb') as f:  # 'rb'는 read binary의 약자
        file = pickle.load(f)
        
    return file

def get_text_by_xpath(driver:webdriver.Chrome, xpath:str):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.text
    except:
        return 'None'
    
def get_driver(use_headless:bool=True, proxy_server=None) -> webdriver.Chrome:
    chrome_options = Options()
    chrome_options.add_argument('--disable-gpu')  # GPU 비활성화
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-extensions')  # 확장 프로그램 비활성화
    chrome_options.add_experimental_option("prefs", {
        "profile.managed_default_content_settings.images": 2,  # 이미지 비활성화
        "profile.default_content_setting_values.notifications": 2,  # 알림 비활성화
        "profile.default_content_setting_values.media_stream": 2,  # 미디어 스트림 비활성화
    })
    
    if use_headless:
        chrome_options.add_argument('--headless=new')  # 최신 Chrome에서 안정적으로 작동
    
    if proxy_server:
        chrome_options.add_argument(f'--proxy-server={proxy_server}')

    # Chrome 드라이버를 자동으로 설정
    service = Service(ChromeDriverManager().install())

    # Chrome 드라이버 로드
    driver = webdriver.Chrome(service=service, options=chrome_options)

    return driver

# 영화 정보 긁어오기 함수 정의

In [27]:
# 왓챠
def get_watch_infos(driver:webdriver.Chrome, movie_id:str, n_comment:int = 10) -> dict:
    """
    driver : chrome driver
    n_comment : 코멘트 개수
    movie_id : 영화의 url id
    """
    watcha_infos = {}
    watcha_infos['describe'] = """
    title : 영화 제목
    moive_info : 영화정보 (연도, 장르, 제작국가)
    movie_info_2 : 영화정보 (상영시간, 제한연령)
    cast_production_info_list : 출연/제작 정보, 감독, 배우 정보
    movie_synopsis : 영화 요약 소개
    avg_rating : 평균 평점
    avg_rating_n : 평점 남긴 숫자
    comments_list: 커멘트 list
    """

    url = f'https://pedia.watcha.com/ko-KR/contents/{movie_id}'
    driver.get(url)

    time.sleep(1)

    # 팝업 창 뜨면 없애기
    try:
        driver.find_element(By.CLASS_NAME, "hsDVweTz").click()
    except:
        pass
        
    time.sleep(1)


    # 제목
    title_xpath = '//*[@id="root"]/div[1]/section/div/div[2]/div/div/div[1]/div[2]/div/h1'
    watcha_infos['title'] = get_text_by_xpath(driver, title_xpath)

    # 영화정보 (연도, 장르, 제작국가)
    movie_info_xpath = '//*[@id="root"]/div[1]/section/div/div[2]/div/div/div[1]/div[2]/div/div[2]'
    watcha_infos['movie_info'] = get_text_by_xpath(driver, movie_info_xpath)
    
    # 영화정보2 (런타임, 연령)
    movie_info_xpath_2 = '//*[@id="root"]/div[1]/section/div/div[2]/div/div/div[1]/div[2]/div/div[3]'
    watcha_infos['movie_info_2'] = get_text_by_xpath(driver, movie_info_xpath_2)


    # 출연/제작 정보
    i = 1
    cast_production_info_list = []

    while True:
        cast_production_info_xpath = f'//*[@id="content_credits"]/section/div[1]/ul/li[{i}]/a/div[2]'
        cast_production_info = get_text_by_xpath(driver, cast_production_info_xpath)

        if cast_production_info != 'None':
            cast_production_info_list.append(cast_production_info)
            i += 1
        else:
            break

    watcha_infos['cast_production_info_list'] = cast_production_info_list

    # 영화 내용 소개
    movie_synopsis_xpath = '//*[@id="root"]/div[1]/section/div/div[2]/div/div/div[2]/section[1]/div[2]/section[3]/p'
    watcha_infos['movie_synopsis'] = get_text_by_xpath(driver, movie_synopsis_xpath)

    # 평균평점
    avg_rating_xpath = '//*[@id="root"]/div[1]/section/div/div[2]/div/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[1]'
    watcha_infos['avg_rating'] = get_text_by_xpath(driver, avg_rating_xpath)

    # 평점수
    avg_rating_xpath_n_xpath = '//*[@id="root"]/div[1]/section/div/div[2]/div/div/div[2]/section[1]/div[1]/section/span/strong'
    watcha_infos['avg_rating_n'] = get_text_by_xpath(driver, avg_rating_xpath_n_xpath)

    # 왓챠피디아 기생충 코멘트 더보기 url
    url = 'https://pedia.watcha.com/ko-KR/contents/mdRL4eL/comments'
    driver.get(url)

    # JavaScript로 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 페이지 로딩 시간을 고려해 잠시 대기

    # 왓챠피디아 기생충 코멘트 더보기 url
    url = f'https://pedia.watcha.com/ko-KR/contents/{movie_id}/comments'
    driver.get(url)

    # JavaScript로 페이지 끝까지 스크롤
    # 한번 스크롤하면 약 9개이상 증가함
    for _ in range(int(n_comment / 9) + 1):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    # comments
    # 1. 스크롤 내려줘야 10개이상 볼 수 있음
    # 2. '보기' 눌러줘야 함
    comments_list = []
    for i in range(1, n_comment+1):
        comment_xpath = f'//*[@id="root"]/div[1]/section/section/div/div/div/ul/div[{i}]'
        comment_button_xpath = f'//*[@id="root"]/div[1]/section/section/div/div/div/ul/div[{i}]/div[2]/a/div/span/button'
        if get_text_by_xpath(driver, comment_button_xpath) == '보기':
            try:
                driver.find_element(By.XPATH,comment_button_xpath).click()
            except:
                continue
        comments_list.append(get_text_by_xpath(driver, comment_xpath))
        
    watcha_infos['comments_list'] = comments_list
    
    return watcha_infos

# 네이버 검색
def get_naver_infos(driver:webdriver.Chrome, movie_name:str) -> dict:
    naver_infos = {}
    naver_infos['describe'] = """
    개요 : 장르, 나라, 시간
    개봉 : 개봉일
    평점 : 네이버 평점
    관객수 or 채널
    """

    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_name}'
    driver.get(url)
    time.sleep(1)


    # 관객수
    i = 1
    while True:
        key_xpath = f'//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/dl/div[{i}]/dt'
        value_xpath = f'//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/dl/div[{i}]/dd'
        key = get_text_by_xpath(driver, key_xpath)
        value = get_text_by_xpath(driver, value_xpath)
        
        if value != 'None':
            naver_infos[key] = value
            i += 1
        else:
            break
    
    if len(naver_infos) == 1:
        # 한번 더 시도
        i = 1
        while True:
            key_xpath = f'//*[@id="main_pack"]/div[3]/div[2]/div[2]/div/div[1]/dl/div[{i}]/dt'
            value_xpath = f'//*[@id="main_pack"]/div[3]/div[2]/div[2]/div/div[1]/dl/div[{i}]/dd'
            key = get_text_by_xpath(driver, key_xpath)
            value = get_text_by_xpath(driver, value_xpath)
            
            if value != 'None':
                naver_infos[key] = value
                i += 1
            else:
                break
            
    return naver_infos

# 위키검색
def get_wiki_infos(driver:webdriver.Chrome, movie_name:str) -> dict:
    def get_movie_info(url):
        driver.get(url)
        time.sleep(1)

        # 영화정보
        movie_info = {}
        i = 2

        # 2가지 경우가 있음
        while True:
            key_xpath = f'//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[{i+1}]/th'
            value_xpath = f'//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr[{i+1}]/td'

            key = get_text_by_xpath(driver, key_xpath)
            value = get_text_by_xpath(driver, value_xpath)
            
            if value != 'None':
                movie_info[key] = value
                i += 1
            else:
                break
        
        # 한번 더 시도
        if len(movie_info) == 0:        
            while True:
                key_xpath = f'//*[@id="mw-content-text"]/div[1]/table/tbody/tr[{i+1}]/th'
                value_xpath = f'//*[@id="mw-content-text"]/div[1]/table/tbody/tr[{i+1}]/td'

                key = get_text_by_xpath(driver, key_xpath)
                value = get_text_by_xpath(driver, value_xpath)
                
                if value != 'None':
                    movie_info[key] = value
                    i += 1
                else:
                    break
            
        return movie_info
        
    wiki_infos = {}
    wiki_infos['describe'] = """
    movie_info : 영화정보 (각본, 제작, 촬영, 편집, 음악, 제작사, 배급사, 개봉일, 시간, 국가, 언어)
    """
    
    # 2가지 버전으로 검색
    # '인턴 (영화)'검색 안되면  '인턴'로 검색해야함
    url = f'https://ko.wikipedia.org/wiki/{movie_name} (영화)'

    movie_info = get_movie_info(url)
    if len(movie_info) == 0:
        url = f'https://ko.wikipedia.org/wiki/{movie_name}'
        
        movie_info = get_movie_info(url)

    wiki_infos['movie_info'] = movie_info

    return wiki_infos

In [28]:
driver = get_driver()

# 왓챠피디아 기생충 url
movie_id = 'mdRL4eL'
movie_name = '기생충'

watcha_infos = get_watch_infos(driver, movie_id)
print(watcha_infos['describe'])

naver_infos = get_naver_infos(driver, movie_name)
print(naver_infos['describe'])

wiki_infos = get_wiki_infos(driver, movie_name)
print(wiki_infos['describe'])

driver.quit()


    title : 영화 제목
    moive_info : 영화정보 (연도, 장르, 제작국가)
    movie_info_2 : 영화정보 (상영시간, 제한연령)
    cast_production_info_list : 출연/제작 정보, 감독, 배우 정보
    movie_synopsis : 영화 요약 소개
    avg_rating : 평균 평점
    avg_rating_n : 평점 남긴 숫자
    comments_list: 커멘트 list
    

    개요 : 장르, 나라, 시간
    개봉 : 개봉일
    평점 : 네이버 평점
    관객수 or 채널
    

    movie_info : 영화정보 (각본, 제작, 촬영, 편집, 음악, 제작사, 배급사, 개봉일, 시간, 국가, 언어)
    


# 왓챠에서 영화 url 구하기

In [7]:
def get_movie_url(driver, movie_name:str) -> str:
    url = f'https://pedia.watcha.com/ko-KR/search?query={movie_name}'
    driver.get(url)
    time.sleep(1)

    # 팝업 창 뜨면 없애기
    try:
        driver.find_element(By.CLASS_NAME, "hsDVweTz").click()
    except:
        pass
    time.sleep(1)

    # 가장 첫번째 창 클릭
    driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/section/section/div[2]/div[1]/section/section[2]/div[1]/ul/li[1]/a/div[1]/div[1]').click()
    time.sleep(1)

    movie_url = driver.current_url

    return movie_url

In [8]:
driver = get_driver()

movie_name = '범죄도시4'
movie_url = get_movie_url(driver, movie_name)
movie_id = movie_url.split('/')[-1]

print(movie_url)
print(movie_id)

driver.quit()

https://pedia.watcha.com/ko-KR/contents/mWyawb6
mWyawb6


In [10]:
# 영화 : 왓챠 영화 id 만들기

driver = get_driver()

movies = ['월플라워',
          '기생충',
          '메멘토',
          '노인을 위한 나라는 없다',
          '파이트 클럽',
          '박화영',
          '거인',
          '끝까지 간다',
          '헤드윅',
          '더 울프 오브 월 스트리트',
          '어바웃타임',
          '포레스트 검프',
          '쇼생크 탈출', 
          '인턴', 
          '글로리 로드', 
          '올드 가드', 
          '돈 룩 업', 
          '머니볼', 
          '레블 리지', 
          '청년경찰',
          '인사이드 아웃 2',
          '엘리멘탈',
          '서울의 봄',
          '컨택트',
          '매드맥스: 분노의 도로',
          '향수 - 어느 살인자의 이야기',
          '탑건: 매버릭',
          '세 얼간이',
          '리미트리스',
          '킹스맨: 시크릿 에이전트',
          '데드풀과 울버린',
          '범죄도시4',
          '범죄도시3', 
          '더 퍼스트 슬램덩크', 
          '탈주', 
          '듄: 파트2', 
          '파일럿', 
          '그녀가 죽었다', 
          '파묘', 
          '웡카', 
          '아가일', 
          '레이니 데이 인 뉴욕', 
          '혹성탈출: 새로운 시대', 
          '콰이어트 플레이스: 첫째 날',
          '타짜',
          '범죄의 재구성',
          '케빈에 대하여',
          '아마데우스',
          '반지의제왕 : 반지 원정대',
          '글레디에이터',
          '광해, 왕이 된 남자',
          '바스터즈: 거친 녀석들',
          '공작',
          '크레이지 스투피드 러브']

movie_id_dict = {}
for movie in tqdm(movies):
    movie_id_dict[movie] = get_movie_url(driver, movie).split('/')[-1]
    
driver.quit()

100%|██████████| 54/54 [03:27<00:00,  3.84s/it]


In [54]:
movie_id_dict

{'월플라워': 'm5Y0qRd',
 '기생충': 'mdRL4eL',
 '메멘토': 'mMO2JAO',
 '노인을 위한 나라는 없다': 'm5XMJ1W',
 '파이트 클럽': 'myWqxp5',
 '박화영': 'mdja1zr',
 '거인': 'm5ZXXm9',
 '끝까지 간다': 'mOPoEMa',
 '헤드윅': 'mJ5xGBd',
 '더 울프 오브 월 스트리트': 'm5DpxB5',
 '어바웃타임': 'mO2x9k5',
 '포레스트 검프': 'mY5QvGW',
 '쇼생크 탈출': 'mNdKBkO',
 '인턴': 'm5NVEE2',
 '글로리 로드': 'mQO8KbW',
 '올드 가드': 'mOgj87g',
 '돈 룩 업': 'mdB7mlP',
 '머니볼': 'mO09Exd',
 '레블 리지': 'm5DPnAy',
 '청년경찰': 'm5nQDa3',
 '인사이드 아웃 2': 'mWw30ZQ',
 '엘리멘탈': 'mOPVwYV',
 '서울의 봄': 'mWvqVXx',
 '컨택트': 'mdRLwlv',
 '매드맥스: 분노의 도로': 'mWzVYBW',
 '향수 - 어느 살인자의 이야기': 'mDWvLZd',
 '탑건: 매버릭': 'mdE1GGd',
 '세 얼간이': 'mOk6rwW',
 '리미트리스': 'm5alkNd',
 '킹스맨: 시크릿 에이전트': 'mO0gZ8X',
 '데드풀과 울버린': 'md6re8q',
 '범죄도시4': 'mWyawb6',
 '범죄도시3': 'mOllQjn',
 '더 퍼스트 슬램덩크': 'mOgjx80',
 '탈주': 'm5nQ1nd',
 '듄: 파트2': 'mWvqG3e',
 '파일럿': 'mdMR79R',
 '그녀가 죽었다': 'mWzwZR0',
 '파묘': 'm53mNG2',
 '웡카': 'mdMRXxl',
 '아가일': 'mWJXGD3',
 '레이니 데이 인 뉴욕': 'm5rQwNE',
 '혹성탈출: 새로운 시대': 'm5YLNbl',
 '콰이어트 플레이스: 첫째 날': 'm5X2Kkp',
 '타짜': 'm5rwLXm',
 '범

In [57]:
movie_id_dict['파일럿'] = 'mOoprrM'
movie_id_dict['파일럿']

'mOoprrM'

In [58]:
save_file_to_pickle(movie_id_dict, './movie_id_dict_241016.pkl')

# 영화 정보 구하기

In [78]:
import re

# 2시간22 -> 144
def time_to_minutes(time_str: str) -> int:
    # '시간'과 '분' 앞에 있는 숫자를 추출하는 정규식
    hours_match = re.search(r'(\d+)시간', time_str)
    minutes_match = re.search(r'(\d+)분', time_str)

    # 추출한 '시간'과 '분' 값을 int로 변환 (없을 경우 0으로 처리)
    hours = int(hours_match.group(1)) if hours_match else 0
    minutes = int(minutes_match.group(1)) if minutes_match else 0

    # 시간을 분으로 변환한 값과 분을 더해줌
    total_minutes = hours * 60 + minutes
    
    return total_minutes

def extract_number(text: str) -> float:
    # 정규식을 이용해 숫자 부분만 추출
    match = re.search(r'(\d+\.\d+|\d+)', text)
    
    if match:
        return float(match.group(0))
    else:
        return None
    
def extract_movie_age(text: str) -> str:
    if '전체' == text:
        return '12'
    else:
        extract_number(text)
    
def separate_cast(cast_list):
    main_cast = []  # 주연 리스트
    supporting_cast = []  # 조연 리스트

    for item in cast_list:
        if '주연' in item:
            # 주연 항목에서 이름만 추출 (이름은 '\n' 앞에 있음)
            name = item.split('\n')[0]
            main_cast.append(name)
        elif '조연' in item:
            # 조연 항목에서 이름만 추출 (이름은 '\n' 앞에 있음)
            name = item.split('\n')[0]
            supporting_cast.append(name)

    return main_cast, supporting_cast

def process_reviews(review_list):
    processed_reviews = []
    
    for review in review_list:
        parts = review.split('\n')
        
        
        # 유저 이름과 평점은 고정된 위치에 있고,
        # 이유는 좋아요 수와 답글 수 사이에 있음.
        user_name = parts[0]  # 첫 번째 요소: 유저 이름

        try:
            rating = float(parts[1])
        except (ValueError, IndexError):
            rating = 'None'

        parts = [part for part in parts if part not in ['', 'None']]
        if '왓챠 소장' in parts[-2] :
            likes = int(parts[-4])
            n_comments = int(parts[-3])
            reason = "\n".join(parts[2:-4])
        else:
            likes = int(parts[-3])
            n_comments = int(parts[-2])
            reason = "\n".join(parts[2:-3]) 

        # rating = float(parts[1])  # 두 번째 요소: 평점
        
        # 마지막 두 요소는 항상 좋아요 수와 답글 수
        # likes = int(parts[-3])  # 좋아요 수는 마지막에서 세 번째
        # n_comments = int(parts[-2])  # 답글 수는 마지막에서 두 번째
        
        # 이유는 나머지 모든 요소를 합쳐서 처리
        # reason = "\n".join(parts[2:-3])  # 2번째부터 마지막 3번째 전까지는 이유

        # 딕셔너리 생성
        review_dict = {
            'user_name': user_name,
            'rating': rating,
            'reason': reason,
            'likes': likes,
            'n_comments': n_comments
        }

        # 결과 리스트에 추가
        processed_reviews.append(review_dict)
    
    return processed_reviews

In [80]:
def process_reviews(review_list):
    processed_reviews = []
    
    for review in review_list:
        parts = review.split('\n')

        # 리뷰 리스트에서 유효한 부분만 필터링 (빈 문자열과 'None'은 제외)
        parts = [part for part in parts if part not in ['', 'None']]
        
        # 필터링 후 데이터가 없는 경우 건너뛴다
        if len(parts) == 0:
            continue
        
        # 유저 이름과 평점은 고정된 위치에 있음 (필터링된 첫 번째 요소)
        user_name = parts[0] if len(parts) > 0 else 'Unknown'

        try:
            rating = float(parts[1]) if len(parts) > 1 else 'None'  # 평점이 있으면 적용
        except (ValueError, IndexError):
            rating = 'None'

        # 좋아요 수와 답글 수를 확인
        if len(parts) >= 4 and '왓챠 소장' in parts[-2]:  # '왓챠 소장'이 포함된 경우
            likes = int(parts[-4]) if len(parts) > 3 else 0
            n_comments = int(parts[-3]) if len(parts) > 3 else 0
            reason = "\n".join(parts[2:-4])  # 이유 추출
        elif len(parts) >= 3:  # 일반적인 경우
            likes = int(parts[-3]) if len(parts) > 2 else 0
            n_comments = int(parts[-2]) if len(parts) > 2 else 0
            reason = "\n".join(parts[2:-3])  # 이유 추출
        else:
            # 데이터가 부족한 경우 기본값으로 설정
            likes = 0
            n_comments = 0
            reason = "\n".join(parts[2:]) if len(parts) > 2 else ''

        # 딕셔너리 생성
        review_dict = {
            'user_name': user_name,
            'rating': rating,
            'reason': reason,
            'likes': likes,
            'n_comments': n_comments
        }

        # 결과 리스트에 추가
        processed_reviews.append(review_dict)
    
    return processed_reviews

In [63]:
def get_movie_info(movie_name, movie_id, watcha_infos, naver_infos, wiki_infos):
    # 후처리
    movie_info = {}
    movie_info['describe'] = """
    *** 왓챠 추출 ***
    movie_name : 검색한 영화 제목 (str)
    movie_id : 왓챠 영화 id (str)
    title : 왓챠 영화 제목 (str)
    year : 연도 (str)
    genre : 장르 (str)
    country : 국가 (str)
    runtime : 상영시간 (str)
    age : 제한연령 (str) , 전체이용가는 12세로 처리
    main_cast : 주연 (list)
    sup_cast : 조연 (list)
    avg_rating : 평균평점 (str)
    avg_rating_n : 평점수 (str)
    comments : 커멘트 (list), [Dict{유저이름, 평점, 좋아요 수, 이유, 답글 수}, Dict, ...]
    
    *** 네이버 추출 ***
    total_audience : 총 관객수 , 만명단위 , 영화진흥위원회 통합 전산망 데이터로 업데이트 되고 있다고 함. 2005년도 전은 없음.
    channel : 상영된 OTT
    
    *** 위치 추출 ***
    writer: 각본
    producer: 제작
    dp: 촬영
    original_story: 원작
    editor: 편집
    music_director: 음악
    production_company: 제작사
    distribution_company: 배급사
    production_budget: 제작비
    box_office_revenue: 흥행수익
    language: 언어
    """
    movie_info['movie_name'] = movie_name
    movie_info['movie_id'] = movie_id 
    movie_info['movie_title'] = watcha_infos['title']

    # 왓챠 후처리
    tmp = watcha_infos['movie_info'].split('·')
    year, genre, country = [item.strip() for item in tmp]

    movie_info['year'] = year
    movie_info['genre'] = genre
    movie_info['country'] = country

    tmp = watcha_infos['movie_info_2'].split('·')
    if len(tmp) == 2:
        runtime, age = [item.strip() for item in tmp]
    elif len(tmp) == 1:
        runtime = tmp[0].strip()
        age = 'None'
    else:
        runtime = 'None'
        age = 'None'
    
    runtime = time_to_minutes(runtime)
    age = extract_movie_age(age)
    movie_info['runtime'] = runtime
    movie_info['age'] = age

    main_cast, sup_cast = separate_cast(watcha_infos['cast_production_info_list'])
    movie_info['main_cast'] = main_cast
    movie_info['sup_cast'] = sup_cast

    movie_info['avg_rating'] = watcha_infos['avg_rating']
    movie_info['avg_rating_n'] = extract_number(watcha_infos['avg_rating_n'])

    movie_info['comments'] = process_reviews(watcha_infos['comments_list'])

    # 네이버 후처리    
    movie_info['channel'] = naver_infos.get('채널', None)
    total_n = naver_infos.get('관객수', None)
    if not total_n:
        movie_info['total_audience'] = None
    else:
        movie_info['total_audience'] = extract_number(total_n)

    # 위키 후처리
    movie_info['writer'] = wiki_infos['movie_info'].get('각본', None)
    movie_info['producer'] = wiki_infos['movie_info'].get('제작', None)
    movie_info['dp'] = wiki_infos['movie_info'].get('촬영', None)
    movie_info['original_story'] = wiki_infos['movie_info'].get('원작', None)
    movie_info['editor'] = wiki_infos['movie_info'].get('편집', None)
    movie_info['music_director'] = wiki_infos['movie_info'].get('음악', None)
    movie_info['production_company'] = wiki_infos['movie_info'].get('제작사', None)
    movie_info['distribution_company'] = wiki_infos['movie_info'].get('배급사', None)
    movie_info['production_budget'] = wiki_infos['movie_info'].get('제작비', None)
    movie_info['box_office_revenue'] = wiki_infos['movie_info'].get('흥행수익', None)
    movie_info['language'] = wiki_infos['movie_info'].get('언어', None)

    return movie_info

In [88]:
movie_id_dict = load_file_from_pickle(r'C:\Users\ose06\OneDrive\바탕 화면\LLMREC\SmallTalk2Rec_Exp-crawling\crawling\movie_id_dict_241016.pkl')
print(movie_id_dict)

movie_names = list(movie_id_dict.keys())

{'월플라워': 'm5Y0qRd', '기생충': 'mdRL4eL', '메멘토': 'mMO2JAO', '노인을 위한 나라는 없다': 'm5XMJ1W', '파이트 클럽': 'myWqxp5', '박화영': 'mdja1zr', '거인': 'm5ZXXm9', '끝까지 간다': 'mOPoEMa', '헤드윅': 'mJ5xGBd', '더 울프 오브 월 스트리트': 'm5DpxB5', '어바웃타임': 'mO2x9k5', '포레스트 검프': 'mY5QvGW', '쇼생크 탈출': 'mNdKBkO', '인턴': 'm5NVEE2', '글로리 로드': 'mQO8KbW', '올드 가드': 'mOgj87g', '돈 룩 업': 'mdB7mlP', '머니볼': 'mO09Exd', '레블 리지': 'm5DPnAy', '청년경찰': 'm5nQDa3', '인사이드 아웃 2': 'mWw30ZQ', '엘리멘탈': 'mOPVwYV', '서울의 봄': 'mWvqVXx', '컨택트': 'mdRLwlv', '매드맥스: 분노의 도로': 'mWzVYBW', '향수 - 어느 살인자의 이야기': 'mDWvLZd', '탑건: 매버릭': 'mdE1GGd', '세 얼간이': 'mOk6rwW', '리미트리스': 'm5alkNd', '킹스맨: 시크릿 에이전트': 'mO0gZ8X', '데드풀과 울버린': 'md6re8q', '범죄도시4': 'mWyawb6', '범죄도시3': 'mOllQjn', '더 퍼스트 슬램덩크': 'mOgjx80', '탈주': 'm5nQ1nd', '듄: 파트2': 'mWvqG3e', '파일럿': 'mOoprrM', '그녀가 죽었다': 'mWzwZR0', '파묘': 'm53mNG2', '웡카': 'mdMRXxl', '아가일': 'mWJXGD3', '레이니 데이 인 뉴욕': 'm5rQwNE', '혹성탈출: 새로운 시대': 'm5YLNbl', '콰이어트 플레이스: 첫째 날': 'm5X2Kkp', '타짜': 'm5rwLXm', '범죄의 재구성': 'm5nQDnd', '케빈에 대하여': 'm5aMDM5', '아마

In [99]:
movie_infos = {}

for movie_name in tqdm(movie_names):
    movie_id = movie_id_dict[movie_name]
    
    driver = get_driver(use_headless=False)

    watcha_infos = get_watch_infos(driver, movie_id)
    naver_infos = get_naver_infos(driver, movie_name)
    wiki_infos = get_wiki_infos(driver, movie_name)

    movie_info = get_movie_info(movie_name, movie_id, watcha_infos, naver_infos, wiki_infos)
    
    movie_infos[movie_name] = movie_info

    driver.quit()

100%|██████████| 54/54 [26:48<00:00, 29.79s/it]


In [101]:
len(movie_infos)

54

In [102]:
save_file_to_pickle(movie_infos, './movie_infos_241016.pkl')

# proxy server 이용하기

In [106]:
import pandas as pd

# 텍스트 파일 경로 설정
file_path = 'Free_Proxy_List.txt'

# 파일을 읽어와 DataFrame으로 변환
df = pd.read_csv(file_path)
df.head()

,ip,anonymityLevel,asn,country,isp,latency,org,port,protocols,speed,upTime,upTimeSuccessCount,upTimeTryCount,updated_at,responseTime
0,171.228.132.62,elite,AS7552,VN,Viettel Corporation,229,VIETEL,10089,http,27889,100,2,2,2024-10-13T14:14:25.664Z,729
1,49.51.244.112,anonymous,AS132203,US,OPHL,200,"Tencent cloud computing (Beijing) Co., Ltd.",8888,http,1289,100,287,287,2024-10-13T14:18:35.157Z,3410
2,209.97.150.167,elite,AS14061,US,"DigitalOcean, LLC",88,"DigitalOcean, LLC",8080,http,20697,80,44,55,2024-10-13T13:11:05.271Z,357
3,165.232.129.150,elite,AS14061,US,"DigitalOcean, LLC",195,"DigitalOcean, LLC",80,http,11487,100,45,45,2024-10-13T13:37:27.570Z,361
4,45.202.198.16,transparent,AS984,US,Octopus Web Solution Inc,197,Waterloo Network Company Limited,3128,http,12715,100,8,8,2024-10-13T13:14:10.956Z,879


In [107]:
df['country'].unique()

array(['VN', 'US', 'TH', 'RU', 'NL', 'MX', 'IQ', 'IN', 'ID', 'HK', 'GB',
       'FR', 'DE', 'CN', 'CA', 'BR', 'BD'], dtype=object)

In [108]:
df[df['country'] == 'Ko']

,ip,anonymityLevel,asn,country,isp,latency,org,port,protocols,speed,upTime,upTimeSuccessCount,upTimeTryCount,updated_at,responseTime


In [109]:
i = 0
row = df.iloc[i]
proxy_server = f"http:{row['ip']}:{row['port']}"

driver = get_driver(use_headless=True, proxy_server=proxy_server)

In [111]:
movie_id_dict = load_file_from_pickle('movie_id_dict_241016.pkl')
print(movie_id_dict)

movie_names = list(movie_id_dict.keys())

{'월플라워': 'm5Y0qRd', '기생충': 'mdRL4eL', '메멘토': 'mMO2JAO', '노인을 위한 나라는 없다': 'm5XMJ1W', '파이트 클럽': 'myWqxp5', '박화영': 'mdja1zr', '거인': 'm5ZXXm9', '끝까지 간다': 'mOPoEMa', '헤드윅': 'mJ5xGBd', '더 울프 오브 월 스트리트': 'm5DpxB5', '어바웃타임': 'mO2x9k5', '포레스트 검프': 'mY5QvGW', '쇼생크 탈출': 'mNdKBkO', '인턴': 'm5NVEE2', '글로리 로드': 'mQO8KbW', '올드 가드': 'mOgj87g', '돈 룩 업': 'mdB7mlP', '머니볼': 'mO09Exd', '레블 리지': 'm5DPnAy', '청년경찰': 'm5nQDa3', '인사이드 아웃 2': 'mWw30ZQ', '엘리멘탈': 'mOPVwYV', '서울의 봄': 'mWvqVXx', '컨택트': 'mdRLwlv', '매드맥스: 분노의 도로': 'mWzVYBW', '향수 - 어느 살인자의 이야기': 'mDWvLZd', '탑건: 매버릭': 'mdE1GGd', '세 얼간이': 'mOk6rwW', '리미트리스': 'm5alkNd', '킹스맨: 시크릿 에이전트': 'mO0gZ8X', '데드풀과 울버린': 'md6re8q', '범죄도시4': 'mWyawb6', '범죄도시3': 'mOllQjn', '더 퍼스트 슬램덩크': 'mOgjx80', '탈주': 'm5nQ1nd', '듄: 파트2': 'mWvqG3e', '파일럿': 'mOoprrM', '그녀가 죽었다': 'mWzwZR0', '파묘': 'm53mNG2', '웡카': 'mdMRXxl', '아가일': 'mWJXGD3', '레이니 데이 인 뉴욕': 'm5rQwNE', '혹성탈출: 새로운 시대': 'm5YLNbl', '콰이어트 플레이스: 첫째 날': 'm5X2Kkp', '타짜': 'm5rwLXm', '범죄의 재구성': 'm5nQDnd', '케빈에 대하여': 'm5aMDM5', '아마

In [166]:
movie_name = movie_names[0]
movie_id = movie_id_dict[movie_name]

watcha_infos = get_watch_infos(driver, movie_id)
print(watcha_infos['describe'])

naver_infos = get_naver_infos(driver, movie_name)
print(naver_infos['describe'])

wiki_infos = get_wiki_infos(driver, movie_name)
print(wiki_infos['describe'])

movie_info = get_movie_info(watcha_infos, naver_infos, wiki_infos)
print(movie_info['describe'])

KeyboardInterrupt: 

# 정리

In [4]:
from assets.utils import *

movie_id_dict = load_file_from_pickle('./data/movie_id_dict_241013.pkl')
print(movie_id_dict)

movie_names = list(movie_id_dict.keys())

{'쇼생크 탈출': 'mNdKBkO', '인턴': 'm5NVEE2', '글로리 로드': 'mQO8KbW', '올드 가드': 'mOgj87g', '돈 룩 업': 'mdB7mlP', '머니볼': 'mO09Exd', '레블 리지': 'm5DPnAy', '청년경찰': 'm5nQDa3', '범죄도시3': 'mOllQjn', '더 퍼스트 슬램덩크': 'mOgjx80', '탈주': 'm5nQ1nd', '듄: 파트2': 'mWvqG3e', '파일럿': 'mdMR79R', '그녀가 죽었다': 'mWzwZR0', '파묘': 'm53mNG2', '웡카': 'mdMRXxl', '아가일': 'mWJXGD3', '레이니 데이 인 뉴욕': 'm5rQwNE', '혹성탈출: 새로운 시대': 'm5YLNbl', '콰이어트 플레이스: 첫째 날': 'm5X2Kkp'}


In [6]:
from tqdm import tqdm
from assets.scraping import get_watch_infos, get_naver_infos
from assets.postprocess import get_movie_info
from assets.utils import get_driver

movie_infos = {}

for movie_name in tqdm(movie_names):
    movie_id = movie_id_dict[movie_name]
    
    driver = get_driver()

    watcha_infos = get_watch_infos(driver, movie_id)
    naver_infos = get_naver_infos(driver, movie_name)
    wiki_infos = get_wiki_infos(driver, movie_name)

    movie_info = get_movie_info(movie_name, movie_id, watcha_infos, naver_infos, wiki_infos)
    
    movie_infos[movie_name] = movie_info

    driver.quit()

100%|██████████| 20/20 [10:19<00:00, 30.96s/it]


In [16]:
# save_file_to_pickle(movie_infos, './data/movie_infos_241014.pkl')

In [18]:
movie_infos = load_file_from_pickle('./data/movie_infos_241014.pkl')
movie_infos['듄: 파트2']

{'describe': '\n    *** 왓챠 추출 ***\n    movie_name : 검색한 영화 제목 (str)\n    movie_id : 왓챠 영화 id (str)\n    title : 왓챠 영화 제목 (str)\n    year : 연도 (str)\n    genre : 장르 (str)\n    country : 국가 (str)\n    runtime : 상영시간 (str)\n    age : 제한연령 (str) , 전체이용가는 12세로 처리\n    main_cast : 주연 (list)\n    sup_cast : 조연 (list)\n    avg_rating : 평균평점 (str)\n    avg_rating_n : 평점수 (str)\n    comments : 커멘트 (list), [Dict{유저이름, 평점, 좋아요 수, 이유, 답글 수}, Dict, ...]\n    \n    *** 네이버 추출 ***\n    total_audience : 총 관객수 , 만명단위 , 영화진흥위원회 통합 전산망 데이터로 업데이트 되고 있다고 함. 2005년도 전은 없음.\n    channel : 상영된 OTT\n    \n    *** 위치 추출 ***\n    writer: 각본\n    producer: 제작\n    dp: 촬영\n    original_story: 원작\n    editor: 편집\n    music_director: 음악\n    production_company: 제작사\n    distribution_company: 배급사\n    production_budget: 제작비\n    box_office_revenue: 흥행수익\n    language: 언어\n    ',
 'movie_name': '듄: 파트2',
 'movie_id': 'mWvqG3e',
 'movie_title': '듄: 파트2',
 'year': '2024',
 'genre': '액션/모험/드라마/SF',
 'country': '미국, 캐나다',
 

In [19]:
movie_infos.keys()

dict_keys(['쇼생크 탈출', '인턴', '글로리 로드', '올드 가드', '돈 룩 업', '머니볼', '레블 리지', '청년경찰', '범죄도시3', '더 퍼스트 슬램덩크', '탈주', '듄: 파트2', '파일럿', '그녀가 죽었다', '파묘', '웡카', '아가일', '레이니 데이 인 뉴욕', '혹성탈출: 새로운 시대', '콰이어트 플레이스: 첫째 날'])